# Physical Object: Canal

The `Canal` class models the flow of water through a canal with a trapezoidal cross-section. It uses Manning's equation to calculate the outflow based on the canal's geometry and the current water level.

### State Variables
- `volume` (float): The current volume of water in the canal (m^3).
- `water_level` (float): The current water level in the canal (m).
- `outflow` (float): The calculated outflow from the canal for the current step (m^3/s).

### Parameters
- `bottom_width` (float): The width of the bottom of the canal (m).
- `length` (float): The length of the canal segment (m).
- `slope` (float): The longitudinal slope of the canal bed (dimensionless).
- `side_slope_z` (float): The slope of the canal sides (z in z:1, horizontal:vertical).
- `manning_n` (float): Manning's roughness coefficient.

## Simulation Example

The following example simulates a canal that receives a variable inflow of water. We can observe how the canal's water level and outflow respond to these changes. The simulation runs for 1000 seconds.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from swp.simulation_identification.physical_objects.canal import Canal
from swp.core.interfaces import State, Parameters

# 1. Define initial state and parameters
initial_state = State(volume=5e5, water_level=5.0, outflow=50)
parameters = Parameters(
    bottom_width=20.0,
    length=5000.0,
    slope=0.0001,
    side_slope_z=2.0,
    manning_n=0.025
)

# 2. Create a Canal instance
canal = Canal(name="main_canal", initial_state=initial_state, parameters=parameters)

# 3. Simulation settings
dt = 1.0
simulation_duration = 1000
num_steps = int(simulation_duration / dt)
history = []

# 4. Run the simulation loop
for t in range(num_steps):
    # Create a variable inflow for demonstration
    # Starts at 50, increases to 150, then decreases
    inflow = 50 + 100 * np.sin(np.pi * t / num_steps)
    
    # The harness would normally call this; we do it manually
    canal.set_inflow(inflow)
    
    # The step action is empty as inflow is handled separately
    current_state = canal.step(action=None, dt=dt)
    
    # Store a copy of the state
    history.append(current_state.copy())

print("Canal simulation complete.")

## Results and Visualization

The plots below show the canal's response to the changing inflow. As the inflow increases, the water level and volume in the canal rise, which in turn increases the outflow according to Manning's equation.

In [ ]:
# Extract data from history
time = [i * dt for i in range(num_steps)]
inflows = [50 + 100 * np.sin(np.pi * t / num_steps) for t in range(num_steps)]
volumes = [h['volume'] for h in history]
water_levels = [h['water_level'] for h in history]
outflows = [h['outflow'] for h in history]

# Create a DataFrame
df = pd.DataFrame({
    'Time': time,
    'Inflow': inflows,
    'Volume': volumes,
    'Water Level': water_levels,
    'Outflow': outflows
})

print(df.head())

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Inflow and Outflow
ax1.plot(df['Time'], df['Inflow'], label='Inflow', color='blue', linestyle='--')
ax1.plot(df['Time'], df['Outflow'], label='Outflow', color='red')
ax1.set_ylabel('Flow Rate (m^3/s)')
ax1.set_title('Canal Flow Dynamics')
ax1.grid(True)
ax1.legend()

# Water Level
ax2.plot(df['Time'], df['Water Level'], label='Water Level', color='green')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Water Level (m)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()